In [28]:
import pandas as pd
import re

In [95]:
filename = 'january2021.csv'
df = pd.read_csv(filename)
df.drop(df.columns[0], axis=1, inplace=True)
df.drop_duplicates(subset=["link"], inplace=True)
df.set_index(['author', 'link'], inplace=True)
df.head()

date  \
author               link                                                               
sochesd              /r/mechmarket/comments/kmna1j/uscah_gmk_cafe_ba...  1.609277e+09   
IOVERCALLHISTIOCYTES /r/mechmarket/comments/kjjzww/ustx_h_vea_clone_...  1.608834e+09   
intuitivezpz         /r/mechmarket/comments/ke777r/au_h_gmk_metavers...  1.608116e+09   
tehbingg             /r/mechmarket/comments/knmmex/sgh_gmk_oblivion_...  1.609405e+09   
                     /r/mechmarket/comments/khjnsb/sgh_sealed_gmk_ca...  1.608565e+09   

                                                                                                                      post  \
author               link                                                                                                    
sochesd              /r/mechmarket/comments/kmna1j/uscah_gmk_cafe_ba...  [TIMESTAMP](https://imgur.com/a/qdWccNO)\n\nHe...   
IOVERCALLHISTIOCYTES /r/mechmarket/comments/kjjzww/ustx_h_vea_clone_...  [Timestamp](https://imgur.com/a/q0AFgBc), incl...   
intuitivezpz         /r/mechmarket/comments/ke777r/au_h_gmk_metavers...  [Timestamp](https://imgur.com/a/KhY386g)\n\nHi...   
tehbingg             /r/mechmarket/comments/knmmex/sgh_gmk_oblivion_...  [Timestamp](https://imgur.com/a/WWvB4ZM)\n\nHe...   
                     /r/mechmarket/comments/khjnsb/sgh_sealed_gmk_ca...  [Timestamp](https://imgur.com/a/IaHGI87)\n\nHi...   

                                                                                                             rawtext  \
author               link                                                                                              
sochesd              /r/mechmarket/comments/kmna1j/uscah_gmk_cafe_ba...                   sold gmk cafe to u/zidtucy   
IOVERCALLHISTIOCYTES /r/mechmarket/comments/kjjzww/ustx_h_vea_clone_...  bought gmk dots from u/iovercallhistiocytes   
intuitivezpz         /r/mechmarket/comments/ke777r/au_h_gmk_metavers...           sold gmk metaverse r2 to u/fluxury   
tehbingg             /r/mechmarket/comments/knmmex/sgh_gmk_oblivion_...           sold gmk cafe base kit to u/iisshy   
                     /r/mechmarket/comments/khjnsb/sgh_sealed_gmk_ca...           sold gmk cafe base kit to u/iisshy   

                                                                                                                     title  
author               link                                                                                                   
sochesd              /r/mechmarket/comments/kmna1j/uscah_gmk_cafe_ba...  [US-CA][H] GMK Cafe Base (New), GMK Cafe Prepa...  
IOVERCALLHISTIOCYTES /r/mechmarket/comments/kjjzww/ustx_h_vea_clone_...  [US-TX] [H] VE.A clone, 1800x2, Rama Darling H...  
intuitivezpz         /r/mechmarket/comments/ke777r/au_h_gmk_metavers...               [AU] [H] GMK Metaverse R2 [W] Paypal  
tehbingg             /r/mechmarket/comments/knmmex/sgh_gmk_oblivion_...  [SG][H] GMK Oblivion v2 (git base, sealed haga...  
                     /r/mechmarket/comments/khjnsb/sgh_sealed_gmk_ca...  [SG][H] SEALED GMK Cafe Base Kit and Rama Ente...

In [96]:
df['post_lower'] = df['post'].str.lower()
df['title_lower'] = df['title'].str.lower()
only_gmk = df[df['post_lower'].str.contains('gmk') | df['title_lower'].str.contains('gmk')].copy()
only_gmk.shape, df.shape

((420, 6), (559, 6))

In [97]:
def split_on_gmk(x):
    a = x.lower().split("gmk")[1]
    return "gmk " + a.split()[0]

only_gmk['product_name'] = only_gmk['rawtext'].apply(split_on_gmk)
only_gmk

date  \
author               link                                                               
sochesd              /r/mechmarket/comments/kmna1j/uscah_gmk_cafe_ba...  1.609277e+09   
IOVERCALLHISTIOCYTES /r/mechmarket/comments/kjjzww/ustx_h_vea_clone_...  1.608834e+09   
intuitivezpz         /r/mechmarket/comments/ke777r/au_h_gmk_metavers...  1.608116e+09   
tehbingg             /r/mechmarket/comments/knmmex/sgh_gmk_oblivion_...  1.609405e+09   
                     /r/mechmarket/comments/khjnsb/sgh_sealed_gmk_ca...  1.608565e+09   
...                                                                               ...   
malteasers           /r/mechmarket/comments/l526zm/usnj_h_pink_wk_kf...  1.611622e+09   
totalrewrite         /r/mechmarket/comments/lcs5r9/eufr_h_duck_viper...  1.612478e+09   
                     /r/mechmarket/comments/l4timd/eufr_h_duck_viper...  1.611598e+09   
aznblahboy           /r/mechmarket/comments/kwpeos/ussch_gmk_taro_gm...  1.610571e+09   
                     /r/mechmarket/comments/kp5g2q/ussch_gmk_fuyu_fa...  1.609621e+09   

                                                                                                                      post  \
author               link                                                                                                    
sochesd              /r/mechmarket/comments/kmna1j/uscah_gmk_cafe_ba...  [TIMESTAMP](https://imgur.com/a/qdWccNO)\n\nHe...   
IOVERCALLHISTIOCYTES /r/mechmarket/comments/kjjzww/ustx_h_vea_clone_...  [Timestamp](https://imgur.com/a/q0AFgBc), incl...   
intuitivezpz         /r/mechmarket/comments/ke777r/au_h_gmk_metavers...  [Timestamp](https://imgur.com/a/KhY386g)\n\nHi...   
tehbingg             /r/mechmarket/comments/knmmex/sgh_gmk_oblivion_...  [Timestamp](https://imgur.com/a/WWvB4ZM)\n\nHe...   
                     /r/mechmarket/comments/khjnsb/sgh_sealed_gmk_ca...  [Timestamp](https://imgur.com/a/IaHGI87)\n\nHi...   
...                                                                                                                    ...   
malteasers           /r/mechmarket/comments/l526zm/usnj_h_pink_wk_kf...  [Timestamp](https://imgur.com/a/4Y2Xezs)\n\nHe...   
totalrewrite         /r/mechmarket/comments/lcs5r9/eufr_h_duck_viper...  [Timestamp](https://imgur.com/a/WaxQtyD)\n\nHi...   
                     /r/mechmarket/comments/l4timd/eufr_h_duck_viper...  [Timestamp](https://imgur.com/a/dkkbk3C)\n\nHi...   
aznblahboy           /r/mechmarket/comments/kwpeos/ussch_gmk_taro_gm...  [Timestamps](https://imgur.com/a/JqtaK8G)  \n ...   
                     /r/mechmarket/comments/kp5g2q/ussch_gmk_fuyu_fa...  [Timestamps](https://imgur.com/a/B0eD3H7)  \n~...   

                                                                                                                   rawtext  \
author               link                                                                                                    
sochesd              /r/mechmarket/comments/kmna1j/uscah_gmk_cafe_ba...                         sold gmk cafe to u/zidtucy   
IOVERCALLHISTIOCYTES /r/mechmarket/comments/kjjzww/ustx_h_vea_clone_...        bought gmk dots from u/iovercallhistiocytes   
intuitivezpz         /r/mechmarket/comments/ke777r/au_h_gmk_metavers...                 sold gmk metaverse r2 to u/fluxury   
tehbingg             /r/mechmarket/comments/knmmex/sgh_gmk_oblivion_...                 sold gmk cafe base kit to u/iisshy   
                     /r/mechmarket/comments/khjnsb/sgh_sealed_gmk_ca...                 sold gmk cafe base kit to u/iisshy   
...                                                                                                                    ...   
malteasers           /r/mechmarket/comments/l526zm/usnj_h_pink_wk_kf...              sold gmk taro + alpacas to u/whoiswoo   
totalrewrite         /r/mechmarket/comments/lcs5r9/eufr_h_duck_viper...               sold gmk first love to /u/snow_weiss   
                     /r/

In [98]:
from collections import defaultdict
all_prices = defaultdict(list)

money_regex = r'\$(\d+)|(\d+)\$'

sets = ['base', 'nov', 'alpha', 'accent', 'bar', 'light base', 'dark base', 'deskmat', 'rama']

sales_data = []

def match_product(df):
    for row in df.itertuples():
        s = row.post_lower.split("\n")
        product = row.product_name.lower()
        #print(row[0][1])
        for l in s:
            low = l.lower()
            if "gmk " in low and ('~~' in low or 'sold' in low):
                a = low.split("gmk ")[1]
                product_name = "gmk " + re.split(r" |~", a)[0]
                
                matches = re.split(money_regex, low)
                temp_data = {}
                if len(matches) > 1:
                    print(f"FULL LINE: {low}")
                    for i in range(0, len(matches)-1, 3):
                        curr_price = int(matches[i+1] if matches[i+1] else matches[i+2])
                        curr_str = matches[i]
                        kits = []
                        removeBase = False
                        empty=True
                        for x in sets:
                            if x in curr_str:
                                empty=False
                                if x == 'nov':
                                    kits.append('novelties')
                                elif x == 'light base' or x == 'dark base':
                                    kits.insert(0, f'{x}')
                                    removeBase = True
                                else:
                                    kits.append(x)
                        products = product_name
                        if i == 0 and not kits:
                            kits = ['base']
                        for x in kits:
                            if removeBase and x == 'base':
                                continue
                            products += f', {x}'
                        
                        if i == 0:
                            temp_data["products"] = products
                            temp_data["str"] = curr_str
                            temp_data["price"] = curr_price
                        
                        if kits and i > 0:
                            print(f"PRODUCT: {temp_data['products']} CURRENT STRING: {temp_data['str']} PRICE: {temp_data['price']}")
                            #all_prices[temp_data['products']].append(temp_data['price'])
                            sales_data.append([row[0][1], temp_data['products'], temp_data['price'], row.date])
                            if temp_data['products'] == 'gmk dark|base, base, novelties':
                                print(row)
                            temp_data["products"] = products
                            temp_data["str"] = curr_str
                            temp_data["price"] = curr_price
                        else:
                            temp_data["price"] = min(temp_data["price"], curr_price)
                        
                    print(f"PRODUCT: {temp_data['products']} CURRENT STRING: {temp_data['str']} PRICE: {temp_data['price']}")
                    #all_prices[temp_data['products']].append(temp_data['price'])
                    sales_data.append([row[0][1], temp_data['products'], temp_data['price'], row.date])

for i, new_df in only_gmk.groupby(level=0):
    match_product(new_df)
    
#print(sales_data)

 uniqkey q:01       | base. alphas and ortho keys are used. shipped in bags.                                                                                                                                                                                |  PRICE: 150
FULL LINE: | gmk laser              | r1 gaijin and blocknet. brand new, still wrapped.                                                                                                                                                                                     | $200 **sold to /u/mase_durp** |
PRODUCT: gmk laser, base CURRENT STRING: | gmk laser              | r1 gaijin and blocknet. brand new, still wrapped.                                                                                                                                                                                     |  PRICE: 200
FULL LINE: |gmk wavez endgame bundle|base, novelties, spacebar, [rama enamel glow, keyforge shishi and orochi.](https://

In [99]:
sales_df = pd.DataFrame(sales_data, columns=['link', 'product', 'price', 'date'])
sales_df['date'] = pd.to_datetime(sales_df['date'], unit = 's')
sales_df.to_csv('results_' + filename)